# Model met data politie

- Laad data politie
- Laad data TK2017
- Koppel data
- Bouw model met opgetelde partijen

In [5]:
import numpy as np
import pandas as pd
import re
import time
#importeer van scikit learn alleen de delenfuncties/klassen die nodig zijn.
from CBSparserOData3 import get_odata, get_top_observations, auto_query, htmlvertaler
from PC4e6_database import *
from sklearn import linear_model, model_selection
#from sklearn.multioutput import MultiOutputRegressor

In [6]:
overlast = 'https://dataderden.cbs.nl/ODataApi/odata/47024NED'
misdrijven = 'https://dataderden.cbs.nl/ODataApi/odata/47022NED'
#misdrijven = 'Misdrijven__soort_misdrijf__plaats_24112020_163412.csv'
#overlast = 'Geregistreerde_overlast_per_buurt_24112020_162212.csv'

https://www.politie.nl/binaries/content/assets/politie/algemeen/data.politie.nl/definities-criminaliteitscijfers-versie-2018.pdf
https://data.politie.nl/portal.html?_la=nl&_catalog=Politie&tableId=47018NED&_theme=84#
https://data.politie.nl/portal.html?_la=nl&_catalog=Politie&tableId=47021NED&_theme=85#

## Ophalen data

In [7]:
TK2017, register = laad_uitslagenPC6("adressen_gl_prioriteit/adressen_TK2017.csv")

In [8]:
TK2017.head(10)

,nummer,lat,lon,stemperc,prioriteit,totaal,opkomst,D66_stemmen,GL_stemmen,PvdA_stemmen,PvdD_stemmen,SP_stemmen,DENK_stemmen,D66_percentage,GL_percentage,PvdA_percentage,PvdD_percentage,SP_percentage,DENK_percentage
PC6,,,,,,,,,,,,,,,,,,,
1011AB,111.0,52.377773,4.905684,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AC,147.0,52.377385,4.908563,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AD,1.0,52.376820,4.904492,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AE,145.0,52.376399,4.910231,0.685179,0.0,6032.0,5.312197,1267.0,1391.0,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AG,98.5,52.375865,4.902948,0.773567,0.0,1082.0,0.814146,226.0,290.0,133.0,116.0,71.0,1.0,0.208872,0.268022,0.122921,0.107209,0.065619,0.000924
1011AH,101.0,52.375694,4.902928,0.773567,0.0,1082.0,0.814146,226.0,290.0,133.0,116.0,71.0,1.0,0.208872,0.268022,0.122921,0.107209,0.065619,0.000924
1011AJ,106.0,52.375424,4.902828,0.773567,0.0,1082.0,0.814146,226.0,290.0,133.0,116.0,71.0,1.0,0.208872,0.268022,0.122921,0.107209,0.065619,0.000924
1011AK,111.0,52.374647,4.903879,0.773567,0.0,1082.0,0.814146,226.0,290.0,133.0,116.0,71.0,1.0,0.208872,0.268022,0.122921,0.107209,0.065619,0.000924
1011AL,5.0,52.368824,4.901983,0.768605,0.0,2081.5,0.828648,425.5,606.0,204.5,198.0,137.5,21.5,0.203295,0.293476,0.101296,0.096078,0.065269,0.009190


In [22]:
# Vervang spaties en aanhalingstekens door een procenteken en het juiste getal
#test = get_odata('https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=99')
test = get_top_observations('https://dataderden.cbs.nl/ODataApi/odata/47015NED', url_filter="$filter=Perioden%20eq%20%272016JJ00%27")
test = auto_query('https://dataderden.cbs.nl/ODataApi/odata/47015NED', "$filter=Perioden%20eq%20%272016JJ00%27")
print(test.shape)
test.head(20)

https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999
https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999
https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=9999
https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=19998
https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=29997
https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=39996
https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=49995
https://dataderden.cbs.nl/ODataApi/odata/47015NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=59994
https://dataderden.cbs.nl/ODataAp

,GeregistreerdeMisdrijven_1,ID,Perioden,Plaatsen,SoortMisdrijf
0,28.0,64,2016JJ00,WP1925,0.0.0
1,24.0,178,2016JJ00,WP2774,0.0.0
2,110.0,292,2016JJ00,WP2145,0.0.0
3,960.0,406,2016JJ00,WP2571,0.0.0
4,18.0,520,2016JJ00,WP1600,0.0.0
5,55.0,634,2016JJ00,WP2626,0.0.0
6,NaN,748,2016JJ00,WP3387,0.0.0
7,429.0,862,2016JJ00,WP1193,0.0.0
8,263.0,976,2016JJ00,WP2673,0.0.0
9,2.0,1090,2016JJ00,WP2394,0.0.0


In [23]:
test.SoortMisdrijf.unique()

array(['0.0.0 ', '1.1.1 ', '1.1.2 ', '1.2.1 ', '1.2.2 ', '1.2.3 ',
       '1.2.4 ', '1.2.5 ', '1.3.1 ', '1.4.1 ', '1.4.2 ', '1.4.3 ',
       '1.4.4 ', '1.4.5 ', '1.4.6 ', '1.4.7 ', '1.5.2 ', '1.6.1 ',
       '1.6.2 ', '1.6.3 ', '2.1.1 ', '2.2.1 ', '2.4.1 ', '2.4.2 ',
       '2.5.1 ', '2.5.2 ', '2.6.1 ', '2.6.2 ', '2.6.3 ', '2.6.4 ',
       '2.6.5 ', '2.6.7 ', '2.6.8 ', '2.6.9 ', '2.6.10', '2.6.11',
       '2.6.12', '2.6.13', '2.6.14', '2.7.2 ', '2.7.3 ', '3.1.1 ',
       '3.1.2 ', '3.1.3 ', '3.2.1 ', '3.2.2 ', '3.3.2 ', '3.3.5 ',
       '3.4.2 ', '3.5.2 ', '3.5.5 ', '3.6.4 ', '3.7.1 ', '3.7.2 ',
       '3.7.3 ', '3.7.4 ', '3.9.1 ', '3.9.2 ', '3.9.3 '], dtype=object)

In [25]:
df_misdrijven = pd.pivot(test, index='Plaatsen', columns='SoortMisdrijf', values='GeregistreerdeMisdrijven_1')
df_misdrijven.head(20)

SoortMisdrijf,0.0.0,1.1.1,1.1.2,1.2.1,1.2.2,1.2.3,1.2.4,1.2.5,1.3.1,1.4.1,...,3.5.2,3.5.5,3.6.4,3.7.1,3.7.2,3.7.3,3.7.4,3.9.1,3.9.2,3.9.3
Plaatsen,,,,,,,,,,,,,,,,,,,,,
WP1000,370.0,18.0,10.0,25.0,3.0,32.0,5.0,4.0,66.0,NaN,...,22.0,1.0,2.0,NaN,1.0,NaN,NaN,19.0,NaN,NaN
WP1001,41.0,8.0,2.0,8.0,1.0,1.0,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
WP1002,179.0,17.0,7.0,8.0,3.0,8.0,1.0,6.0,19.0,NaN,...,8.0,3.0,1.0,NaN,NaN,NaN,1.0,12.0,1.0,1.0
WP1003,138.0,12.0,1.0,12.0,2.0,5.0,2.0,1.0,22.0,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN
WP1004,37.0,1.0,NaN,2.0,NaN,2.0,NaN,2.0,12.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,1.0
WP1005,4877.0,640.0,87.0,379.0,39.0,551.0,62.0,84.0,449.0,NaN,...,117.0,22.0,26.0,2.0,NaN,NaN,2.0,269.0,3.0,15.0
WP1006,571.0,53.0,10.0,67.0,23.0,74.0,4.0,6.0,55.0,NaN,...,20.0,3.0,NaN,NaN,NaN,NaN,1.0,57.0,2.0,1.0
WP1007,2099.0,78.0,32.0,33.0,7.0,337.0,31.0,16.0,161.0,NaN,...,59.0,7.0,9.0,3.0,NaN,NaN,8.0,124.0,1.0,10.0
WP1008,132.0,2.0,1.0,6.0,2.0,15.0,NaN,7.0,13.0,NaN,...,4.0,2.0,1.0,NaN,NaN,NaN,NaN,16.0,NaN,1.0


In [26]:
test =  auto_query(overlast, "$filter=Perioden%20eq%20%272016JJ00%27")
test.head(30)

https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999
https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=9999
https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=19998
https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=29997
https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=39996
https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=49995
https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=59994
https://dataderden.cbs.nl/ODataApi/odata/47024NED/TypedDataSet?$filter=Perioden%20eq%20%272016JJ00%27&$top=9999&$skip=69993
https://dataderden.cb

,Gemeentenaam_2,GeregistreerdeOverlast_1,ID,Overlast,Perioden,SoortRegio_3,WijkenEnBuurten
0,Nederland,333392.0,64,0.0.0,2016JJ00,Land,NL00
1,Aa en Hunze,334.0,178,0.0.0,2016JJ00,Gemeente,GM1680
2,Aa en Hunze,23.0,292,0.0.0,2016JJ00,Wijk,WK168000
3,Aa en Hunze,23.0,406,0.0.0,2016JJ00,Buurt,BU16800000
4,,NaN,520,0.0.0,2016JJ00,,BU16800009
5,Aa en Hunze,34.0,634,0.0.0,2016JJ00,Wijk,WK168001
6,Aa en Hunze,30.0,748,0.0.0,2016JJ00,Buurt,BU16800100
7,Aa en Hunze,4.0,862,0.0.0,2016JJ00,Buurt,BU16800109
8,Aa en Hunze,3.0,976,0.0.0,2016JJ00,Wijk,WK168002
9,Aa en Hunze,1.0,1090,0.0.0,2016JJ00,Buurt,BU16800200


In [30]:
print(test.Overlast.unique())
df_overlast = pd.pivot(test, index='WijkenEnBuurten', columns='Overlast', values='GeregistreerdeOverlast_1')
df_overlast.head(10)

['0.0.0 ' 'E33   ' 'E35   ' 'E38   ' 'E41   ' 'F010  ' 'M061  ' 'M062  '
 'M22   ']


Overlast,0.0.0,E33,E35,E38,E41,F010,M061,M062,M22
WijkenEnBuurten,,,,,,,,,
BU00000000,74942.0,74934.0,2.0,1.0,1.0,1.0,2.0,NaN,1.0
BU00030000,61.0,NaN,51.0,3.0,NaN,NaN,1.0,NaN,6.0
BU00030001,38.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,31.0
BU00030002,110.0,NaN,66.0,7.0,NaN,2.0,NaN,1.0,34.0
BU00030007,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
BU00030008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BU00030009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BU00100101,52.0,NaN,20.0,17.0,NaN,4.0,5.0,NaN,6.0
BU00100202,7.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0


In [42]:
postcodes_plus = ('Data/index_plaatsen.csv', ';')
df_postcodes = pd.read_csv(postcodes_plus[0], sep = postcodes_plus[1]).set_index('PC6')
df_postcodes = df_postcodes.join(TK2017, how='right')
df_postcodes.head()

,Description,Key,Title,Buurt2019,Wijk2019,Gemeente2019,nummer,lat,lon,stemperc,...,PvdA_stemmen,PvdD_stemmen,SP_stemmen,DENK_stemmen,D66_percentage,GL_percentage,PvdA_percentage,PvdD_percentage,SP_percentage,DENK_percentage
PC6,,,,,,,,,,,,,,,,,,,,,
1011AB,NaN,BU03630400,Oosterdokseiland,BU03630400,WK036304,GM0363,111.0,52.377773,4.905684,0.685179,...,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AC,NaN,BU03630400,Oosterdokseiland,BU03630400,WK036304,GM0363,147.0,52.377385,4.908563,0.685179,...,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AD,NaN,BU03630400,Oosterdokseiland,BU03630400,WK036304,GM0363,1.0,52.376820,4.904492,0.685179,...,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AE,NaN,BU03630400,Oosterdokseiland,BU03630400,WK036304,GM0363,145.0,52.376399,4.910231,0.685179,...,414.0,411.0,359.0,291.0,0.210046,0.230603,0.068634,0.068137,0.059516,0.048243
1011AG,NaN,BU03630403,Lastage,BU03630403,WK036304,GM0363,98.5,52.375865,4.902948,0.773567,...,133.0,116.0,71.0,1.0,0.208872,0.268022,0.122921,0.107209,0.065619,0.000924


In [45]:
test = df_overlast.merge(df_postcodes, how='right', left_index=True, right_on='Buurt2019')
test.shape
test.columns

Index(['0.0.0 ', 'E33   ', 'E35   ', 'E38   ', 'E41   ', 'F010  ', 'M061  ',
       'M062  ', 'M22   ', 'Description', 'Key', 'Title', 'Buurt2019',
       'Wijk2019', 'Gemeente2019', 'nummer', 'lat', 'lon', 'stemperc',
       'prioriteit', 'totaal', 'opkomst', 'D66_stemmen', 'GL_stemmen',
       'PvdA_stemmen', 'PvdD_stemmen', 'SP_stemmen', 'DENK_stemmen',
       'D66_percentage', 'GL_percentage', 'PvdA_percentage', 'PvdD_percentage',
       'SP_percentage', 'DENK_percentage'],
      dtype='object')

## Overlast